# **일단 사용해보기**

## **1.환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(langchain)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [ ]:
# 경로 : /content/drive/MyDrive/langchain/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/langchain/requirements.txt

#### 2) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA  #← RetrievalQA를 가져오기
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import WikipediaRetriever

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (3) OpenAI API Key 확인

In [ ]:
def load_api_key(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

path = '/content/drive/MyDrive/langchain/'

# API 키 로드 및 환경변수 설정
openai.api_key = load_api_key(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

* ⚠️ 아래 코드셀은, 실행해서 key가 제대로 보이는지 확인하고 삭제하세요.

In [ ]:
print(os.environ['OPENAI_API_KEY'])

## **2.ChatGPT 사용해보기**

### (1) API 사용하기

#### 1) 채팅을 위한 함수 생성

In [ ]:
def ask_chatgpt1(question):

    # api key 지정
    client = OpenAI()

    # # API를 사용하여 'gpt-3.5-turbo' 모델로부터 응답을 생성합니다.
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},  # 기본 역할 부여
            {"role": "user", "content": question},                          # 질문
        ]
    )

    return response.choices[0].message.content

#### 2) 사용하기

In [ ]:
question = "세계에서 가장 높은 산은 무엇인가요?"
response = ask_chatgpt1(question)
print(response)

#### 😀실습
* 몇가지 질문을 던지고 답변을 받아 봅시다.

### (2) 역할 부여하기

#### 1) 채팅을 위한 함수 생성

In [ ]:
def ask_chatgpt2(sys_role, question):

    client = OpenAI(api_key = api_key)

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": sys_role},
            {"role": "user", "content": question},
        ]
    )

    return response.choices[0].message.content

#### 2) 사용하기

In [ ]:
sys_role = '당신은 아름답고 감동적인 시를 창작하는 데 영감을 주는 시적인 천재입니다. 당신의 시는 감정의 깊이, 자연의 아름다움, 인간 경험의 복잡성을 탐구하는 작품이며, 당신의 시를 읽는 이들의 마음을 움직입니다.'

question = "생성형 AI란 주제로 시를 지어줘. 운율에 맞춰서 작성해줘야 해."
response = ask_chatgpt2(sys_role, question)
print(response)

#### 😀실습
* 역할을 부여한 후 질문하여 답변을 받아 옵시다.
    * 역할 부여 사례 : 여행 플래너, 데이터분석가, 법률가, 건강관리사 등
* 2가지 이상 역할에 대해 실습해 봅시다.


## **3.랭체인 사용해보기**

### (1) 위키피디아 데이터 준비

In [ ]:
wiki_retriever = WikipediaRetriever(lang = "ko")

### (2) 모델준비

In [ ]:
chat = ChatOpenAI(model = "gpt-3.5-turbo")

### (3) 연결(Chain)

In [ ]:
wiki_chat = RetrievalQA.from_llm(llm = chat, retriever = wiki_retriever,
                             return_source_documents=True) # 정보를 가져온 원본 문서를 반환

### (4) 질문답변

* 아래 질문 부분을 수정해서 시도해 봅시다.

In [ ]:
result = wiki_chat("지미카터 대통령의 출신 대학은?")
print(result["result"])